In [453]:
#Chargement des packages
import numpy as np 
import pandas as pd 
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
import seaborn as sns
import matplotlib.pyplot as plt

In [454]:
client = bigquery.Client(project='transformed-data')
project = client.project

/Users/marouan.korfed/miniconda3/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [455]:
query_base_sub_totale = """
SELECT
* except(previous_claims, previous_insurers, previous_terminations, previous_license_cancellations_suspensions)
FROM `mart_insurance.mart_submissions`
            """

In [456]:
query_base_sub_claims = """
WITH submissions AS (
  SELECT job_number, previous_claims
  FROM `transformed-data.mart_insurance.mart_submissions`
)
SELECT job_number, previous_claims.*
FROM submissions,
UNNEST(previous_claims) as previous_claims
ORDER BY 1
            """

In [457]:
query_job_base_sub_totale = client.query(query_base_sub_totale)
base_sub_totale = query_job_base_sub_totale.to_dataframe()

In [458]:
query_job_base_sub_claims = client.query(query_base_sub_claims)
base_sub_claims = query_job_base_sub_claims.to_dataframe()

In [459]:
df_claims_counts = base_sub_claims['job_number'].value_counts().reset_index()
df_claims_counts.columns = ['job_number', 'nb_sinistres']
base_sub_totale = base_sub_totale.merge(df_claims_counts[['job_number','nb_sinistres']],on='job_number', how ='left')
base_sub_totale['nb_sinistres'].fillna(value=0, inplace = True)
base_sub_totale['nb_sinistres'] = base_sub_totale['nb_sinistres'].apply(lambda x: int(x))

In [460]:
# Traitement des dates :
base_sub_totale['period_start']=pd.to_datetime(pd.to_datetime(base_sub_totale['period_start']).dt.date)
base_sub_totale['md_license_date']= pd.to_datetime(base_sub_totale['md_license_date'])
base_sub_totale['md_birthdate']= pd.to_datetime(base_sub_totale['md_birthdate'])
base_sub_totale['vh_insurance_end_date']= pd.to_datetime(base_sub_totale['vh_insurance_end_date'])
base_sub_totale['vh_circulation_date']= pd.to_datetime(base_sub_totale['vh_circulation_date'])
base_sub_totale['submission_created_at']= pd.to_datetime(base_sub_totale['submission_created_at'])

# Ajout : Age du permis, âge du conducteur, âge du véhicule à la date de début d'assurance
base_sub_totale['md_license_seniority']= (base_sub_totale['period_start']-base_sub_totale['md_license_date'])/np.timedelta64(1, 'Y') #age permis en mois
base_sub_totale['md_age']= (base_sub_totale['period_start']-base_sub_totale['md_birthdate'])/np.timedelta64(1, 'Y')
base_sub_totale['age_vehicule']= (base_sub_totale['period_start']-base_sub_totale['vh_circulation_date'])/np.timedelta64(1, 'Y') 
base_sub_totale['jours_derniere_assurance']= (base_sub_totale['period_start']-base_sub_totale['vh_insurance_end_date'])/np.timedelta64(1, 'D')

base_sub_totale.shape

base_sub_totale['submission_created_at'] = base_sub_totale['submission_created_at'].dt.strftime('%Y-%m-%d')

In [461]:
base_sub_totale['md_crm_coefficient']=base_sub_totale['md_crm_coefficient'].astype(float)

In [462]:
db = base_sub_totale
db

,submission_created_at,policy_period_public_id,policy_id,policy_public_id,sf_external_contact_id,policy_term_public_id,job_id,wakam_quote_id,wakam_pricing_version,wakam_price_test,...,policy_period_end,is_last_policy_per_period,is_last_policy_period_bound_per_policy,term_number,payment_plan,nb_sinistres,md_license_seniority,md_age,age_vehicule,jours_derniere_assurance
0,2023-04-05,pc:SeqeuD1GkWHwg6LKH4Ay2,2901,pc:S9NIAYecKt_LndMmKpuOZ,e82794ee-bc7c-e472-2df3-305432a176e5,pc:SjAUpROoHC-TT-WpF5ac-,2901,5e1b1dec-bb22-4cc5-9f3b-60e445cdf051,V1,<NA>,...,2023-04-25 00:01:00+00:00,False,<NA>,1,Plan de paiement mensuel (acompte de 2 mois),0,14.547945,34.147945,22.326027,-251.0
1,2023-04-05,pc:SRnHlbXuxrnu5mqiohXr2,3001,pc:SiFIiLlRmxWIt0MLYyfh2,d4c81bd9-d209-7961-2e0d-fbb0e17f4632,None,3002,ff4d01d8-cc0d-4514-8242-1af6aef492a5,V1,False,...,NaT,<NA>,<NA>,<NA>,None,0,4.668493,34.331507,1.665753,NaN
2,2023-04-05,pc:Scbu-Xi-3LqrfTvCe_Oyf,3002,pc:S9l5GdmxUC0g-gvIVIL3O,d4c81bd9-d209-7961-2e0d-fbb0e17f4632,None,3003,9218ccf1-581d-43f6-aea2-0f7b41aa0b45,V1,False,...,NaT,<NA>,<NA>,<NA>,None,0,4.668493,34.331507,1.665753,NaN
3,2023-04-05,pc:S0LzvB7sz0FCEAqecYlfA,3003,pc:SD4YoViXhytpCb-YaztFs,d4c81bd9-d209-7961-2e0d-fbb0e17f4632,None,3004,31f9570a-a9e0-41e0-b418-73580f959e6a,V1,False,...,NaT,<NA>,<NA>,<NA>,None,0,4.668493,34.331507,1.665753,NaN
4,2023-04-05,pc:SpNb2CFJEVEYaE2FOTtU0,3004,pc:S5n2yKA4ztHqhNrSfpBmx,d4c81bd9-d209-7961-2e0d-fbb0e17f4632,None,3005,2e33b8b1-42de-4a90-816e-f009a9373036,V1,False,...,NaT,<NA>,<NA>,<NA>,None,0,4.668493,34.331507,1.665753,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178318,2023-06-21,pc:SOeHf8j9Z4t3hQDw9eoit,187775,pc:SXT5exCmNpNrobZDDMTSG,None,None,193737,None,None,<NA>,...,NaT,<NA>,<NA>,<NA>,None,0,NaN,NaN,9.569863,1130.0
178319,2023-06-21,pc:SN_W3zdomhOLgZEvUhg1H,187776,pc:SsrR__juYDrn0EYpZTYVc,None,None,193738,None,None,<NA>,...,2024-07-26 00:01:00+00:00,True,<NA>,<NA>,None,0,NaN,NaN,11.824658,NaN
178320,2023-06-21,pc:S3wi-kHOinbMhQvtqFkEh,187777,pc:S3W6YTuCpzz9JMldDxwBN,None,None,193739,None,None,<NA>,...,2024-06-22 00:01:00+00:00,False,<NA>,<NA>,None,0,NaN,NaN,9.476712,1096.0
178321,2023-06-21,pc:SaM6P7ccJ4oK0wESbo-bd,187778,pc:SiQTcObb17DetodM8PYaV,None,None,193740,None,None,<NA>,...,NaT,<NA>,<NA>,<NA>,None,0,NaN,NaN,33.191781,NaN


In [463]:
db['is_tarifs_vus']= (db['has_formulas']==True)
db['is_contrats']= db['bind_date'].notnull().fillna(False)
db['is_MER'] = (db['is_from_aggregator']==True) & (~(db['ph_first_name'].apply(lambda x: "anonymous" in x)))

In [464]:
bins = [17.999, 21.999, 30.999, 50.999, float('inf')]
labels = ['18-21 ans','22-30 ans','31-50 ans','>50 ans']
db['categorie_md_age'] = pd.cut(db['md_age'], bins = bins, labels = labels, right = False)

In [465]:
bins = [0, 5.999, 10.999, 15.999, 20.999, float('inf')]
labels = ['0-5 ans','6-10 ans','11-15 ans','16-20 ans','>20 ans']
db['categorie_vh_age'] = pd.cut(db['age_vehicule'], bins = bins, labels = labels, right = False)

In [466]:
bins = [-1, 0.01, 1.01, float('inf')]
labels = ['0','1','2+']
db['categorie_nb_sinistres'] = pd.cut(db['nb_sinistres'], bins = bins, labels = labels, right = False)

In [467]:
bins = [0, 1.999, 3.999, 5.999, 10.999, 13.999, 20.999, 30.999, 40.999, 50.999, float('inf')]
labels = ['0-1 ans','2-3 ans','4-5 ans','6-10 ans','11-13 ans', '14-20 ans','21-30 ans', '31-40 ans', '41-50 ans', '>50 ans']
db['categorie_md_license_seniority'] = pd.cut(db['md_license_seniority'], bins = bins, labels = labels, right = False)

In [468]:
bins = [0, 0.50000001, 0.60000001, 0.7000001, 0.800000001,0.90000001, 0.9999999, 1.00000000001, float('inf')]
labels = ['0.5','0.51-0.6','0.61-0.7','0.71-0.8','0.81-0.9','0.91-0.99', '1', '1+']
db['categorie_md_crm_coefficient'] = pd.cut(db['md_crm_coefficient'], bins = bins, labels = labels, right = False)

In [469]:
df_organique = db.loc[db['is_from_aggregator']==False]

In [470]:
df_lelynx = db.loc[(db['is_from_aggregator']==True) & (db['utm_campaign']=='lelynx')] 

In [487]:
variables = ["policy_risk_profile", "categorie_md_age", "categorie_vh_age", "categorie_nb_sinistres", "categorie_md_license_seniority", "categorie_md_crm_coefficient"]  

def ReportingFootprintConversion(date1,date2,db,type,variable):
    df1 = db.loc[db['submission_created_at']==date1]
    df2 = db.loc[db['submission_created_at']==date2]
    
    total_counts_df1 = df1.groupby(variable).size()
    tarifs_vus_counts_df1 = df1[df1['is_tarifs_vus']==True].groupby(variable).size()
    tarifs_vus_percentage_df1 = 100*tarifs_vus_counts_df1 / total_counts_df1
    contrats_counts_df1 = df1[df1['is_contrats']==True].groupby(variable).size()
    contrats_percentage_df1 = 100*contrats_counts_df1/ tarifs_vus_counts_df1
    total_counts_df2 = df2.groupby(variable).size()
    tarifs_vus_counts_df2 = df2[df2['is_tarifs_vus']==True].groupby(variable).size()
    tarifs_vus_percentage_df2 = 100*tarifs_vus_counts_df2 / total_counts_df2
    contrats_counts_df2 = df2[df2['is_contrats']==True].groupby(variable).size()
    contrats_percentage_df2 = 100*contrats_counts_df2/ tarifs_vus_counts_df2
    result_df = pd.concat([pd.DataFrame(total_counts_df1),pd.DataFrame(total_counts_df2), pd.DataFrame(tarifs_vus_percentage_df1), pd.DataFrame(tarifs_vus_percentage_df2),pd.DataFrame(contrats_percentage_df1), pd.DataFrame(contrats_percentage_df2)], axis = 1 , join = 'outer', keys=variable)
    result_df.columns = ['App. Tarifaires J' ,'App. Tarif. J-7', 'Footprint (%) J', 'Ft J-7', 'Tx conversion (%) J', 'Tx conv. J-7']
    result_df = result_df.round(decimals=0).reset_index()

    
    total = "Total"
    totalAP_1 = df1[variable].count()
    totalAP_2 = df2[variable].count()
    ft_1 = 100*df1[df1['is_tarifs_vus']==True][variable].count()/df1[variable].count()
    ft_2 = 100*df2[df2['is_tarifs_vus']==True][variable].count()/df2[variable].count()
    conv_1 = 100*df1[df1['is_contrats']==True][variable].count()/df1[df1['is_tarifs_vus']==True][variable].count()
    conv_2 = 100*df2[df2['is_contrats']==True][variable].count()/df2[df2['is_tarifs_vus']==True][variable].count()

    ligne =pd.DataFrame([[total, totalAP_1,
             totalAP_2,
             ft_1, 
             ft_2,
             conv_1,
             conv_2]], columns =[variable,'App. Tarifaires J' ,'App. Tarif. J-7', 'Footprint (%) J', 'Ft J-7', 'Tx conversion (%) J', 'Tx conv. J-7'])

    df_total = pd.concat([result_df, ligne], axis=0)
    df_total = df_total.round(decimals=0)
    df_total=df_total.reset_index(drop = True)

    #https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html
    styler = df_total.style
    (styler.format_index(escape="latex", axis=1)
          .format(subset="Footprint (%) J", precision=0)
          .format(subset="Ft J-7", precision=0)
          .format(subset="Tx conversion (%) J", precision=0)
          .format(subset="Tx conv. J-7", precision=0)
          .format_index(escape="latex", axis=0) #precision
          .hide(level=0, axis=0))
    #def rating_color(v):
     #   if v == "Buy": color = "#33FF85"
      #  elif v == "Sell": color = "#FF5933"
       # else: color = "#FFDD33"
        #return f"color: {color}; font-weight: bold;"
    #(styler.background_gradient(cmap="Oranges", subset="Ft J-7"))
    #(styler.applymap(color_diff, subset = pd.IndexSlice[:,'Footprint (%) J']))
    #.background_gradient(cmap="Oranges", subset="Diff Tx conv."))
    #styler.applymap_index()   lambda v: "rotatebox:{45}--rwrap--latex;", level=2, axis=1)
    with open(r'/Users/marouan.korfed/Desktop/Pricing/Dashboard_LatexV2/Reporting_'+f'{variable}_'+f'{date1}_'+f'{type}''.tex', "w") as f:
      print(styler.to_latex(
        caption="Comparaison Footprint / conversion - "+f'{variable}',
        environment ='table*',
        clines="skip-last;data",
        convert_css=True,
        position_float="centering",
        multicol_align="|c|",
        hrules=True,
    )  , file=f)
    
    return df_total

In [488]:
date1 ="2023-06-20"
date2 ="2023-06-13"
df = df_organique

In [489]:
ReportingFootprintConversion(date1,date2,df,'organique', "policy_risk_profile")
ReportingFootprintConversion(date1,date2,df,'organique', "categorie_md_age")
ReportingFootprintConversion(date1,date2,df,'organique', "categorie_vh_age")
ReportingFootprintConversion(date1,date2,df,'organique', "categorie_nb_sinistres")
ReportingFootprintConversion(date1,date2,df,'organique', "categorie_md_license_seniority")
ReportingFootprintConversion(date1,date2,df,'organique', "categorie_md_crm_coefficient")

,categorie_md_crm_coefficient,App. Tarifaires J,App. Tarif. J-7,Footprint (%) J,Ft J-7,Tx conversion (%) J,Tx conv. J-7
0,0.5,129,112,66.0,60.0,9.0,10.0
1,0.51-0.6,48,65,58.0,71.0,0.0,11.0
2,0.61-0.7,60,46,67.0,65.0,8.0,3.0
3,0.71-0.8,78,133,56.0,68.0,2.0,8.0
4,0.81-0.9,95,110,73.0,65.0,3.0,6.0
5,0.91-0.99,108,135,63.0,71.0,4.0,7.0
6,1,639,654,64.0,68.0,3.0,6.0
7,1+,38,29,0.0,0.0,NaN,NaN
8,Total,1195,1284,62.0,66.0,4.0,7.0


In [490]:
ReportingFootprintConversion(date1,date2,df_lelynx,'lelynx', "policy_risk_profile")
ReportingFootprintConversion(date1,date2,df_lelynx,'lelynx', "categorie_md_age")
ReportingFootprintConversion(date1,date2,df_lelynx,'lelynx', "categorie_vh_age")
ReportingFootprintConversion(date1,date2,df_lelynx,'lelynx', "categorie_nb_sinistres")
ReportingFootprintConversion(date1,date2,df_lelynx,'lelynx', "categorie_md_license_seniority")
ReportingFootprintConversion(date1,date2,df_lelynx,'lelynx', "categorie_md_crm_coefficient")

,categorie_md_crm_coefficient,App. Tarifaires J,App. Tarif. J-7,Footprint (%) J,Ft J-7,Tx conversion (%) J,Tx conv. J-7
0,0.5,2165,2289,68.0,66.0,0.0,0.0
1,0.51-0.6,486,505,66.0,69.0,0.0,0.0
2,0.61-0.7,404,442,64.0,66.0,0.0,0.0
3,0.71-0.8,798,866,59.0,60.0,0.0,0.0
4,0.81-0.9,734,809,52.0,57.0,0.0,1.0
5,0.91-0.99,569,495,53.0,50.0,1.0,1.0
6,1,1684,2135,56.0,51.0,1.0,0.0
7,1+,257,212,0.0,0.0,NaN,NaN
8,Total,7097,7753,58.0,58.0,0.0,0.0


In [491]:
ReportingFootprintConversion(date1,date2,db,'total', "policy_risk_profile")
ReportingFootprintConversion(date1,date2,db,'total', "categorie_md_age")
ReportingFootprintConversion(date1,date2,db,'total', "categorie_vh_age")
ReportingFootprintConversion(date1,date2,db,'total', "categorie_nb_sinistres")
ReportingFootprintConversion(date1,date2,db,'total', "categorie_md_license_seniority")
ReportingFootprintConversion(date1,date2,db,'total', "categorie_md_crm_coefficient")

,categorie_md_crm_coefficient,App. Tarifaires J,App. Tarif. J-7,Footprint (%) J,Ft J-7,Tx conversion (%) J,Tx conv. J-7
0,0.5,2294,2401,68.0,66.0,1.0,1.0
1,0.51-0.6,534,570,65.0,69.0,0.0,1.0
2,0.61-0.7,464,488,64.0,66.0,1.0,0.0
3,0.71-0.8,876,999,59.0,61.0,1.0,1.0
4,0.81-0.9,829,919,54.0,58.0,1.0,1.0
5,0.91-0.99,677,630,55.0,55.0,1.0,3.0
6,1,2323,2789,58.0,55.0,1.0,2.0
7,1+,295,241,0.0,0.0,NaN,NaN
8,Total,8292,9037,59.0,59.0,1.0,1.0


In [492]:
variables = ["policy_risk_profile", "categorie_md_age", "categorie_vh_age", "categorie_nb_sinistres", "categorie_md_license_seniority", "categorie_md_crm_coefficient"]  

def ReportingRepartitionMIX(date1,date2,db,type,variable):
    df1 = db.loc[db['submission_created_at']==date1]
    df2 = db.loc[db['submission_created_at']==date2]
    
    total_counts_df1 = df1.groupby(variable).size()
    total_percentage_df1 = (100*total_counts_df1/total_counts_df1.sum()) 
    tarifs_vus_counts_df1 = df1[df1['is_tarifs_vus']==True].groupby(variable).size()
    tarifs_vus_percentage_df1 = (100*tarifs_vus_counts_df1/tarifs_vus_counts_df1.sum()) 
    contrats_counts_df1 = df1[df1['is_contrats']==True].groupby(variable).size()
    contrats_percentage_df1 = (100*contrats_counts_df1/contrats_counts_df1.sum()) 
    total_counts_df2 = df2.groupby(variable).size()
    total_percentage_df2 = (100*total_counts_df2/total_counts_df2.sum()) 
    tarifs_vus_counts_df2 = df2[df2['is_tarifs_vus']==True].groupby(variable).size()
    tarifs_vus_percentage_df2 = (100*tarifs_vus_counts_df2/tarifs_vus_counts_df2.sum()) 
    contrats_counts_df2 = df2[df2['is_contrats']==True].groupby(variable).size()
    contrats_percentage_df2 = (100*contrats_counts_df2/contrats_counts_df2.sum()) 

    result_df = pd.concat([pd.DataFrame(total_percentage_df1),pd.DataFrame(total_percentage_df2), pd.DataFrame(tarifs_vus_percentage_df1), pd.DataFrame(tarifs_vus_percentage_df2),pd.DataFrame(contrats_percentage_df1), pd.DataFrame(contrats_percentage_df2)], axis = 1 , join = 'outer', keys=variable)
    result_df.columns = ['App. Tarifaires J (%)','App. Tarif. J-7 (%)', 'Tarifs Vus J (%)', 'Tarifs Vus J-7 (%)', 'Contrats J (%)', 'Contrats J-7 (%)']
    
    result_df = result_df.round(decimals=0).reset_index()
  
    total = "Total"

    ligne =pd.DataFrame([[total, 100,
             100,
             100, 
             100,
             100,
             100]], columns =[variable,'App. Tarifaires J (%)','App. Tarif. J-7 (%)', 'Tarifs Vus J (%)', 'Tarifs Vus J-7 (%)', 'Contrats J (%)', 'Contrats J-7 (%)'])

    df_total = pd.concat([result_df, ligne], axis=0)
    df_total = df_total.round(decimals=0)
    df_total=df_total.reset_index(drop = True)


    #https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html
    styler = df_total.style
    (styler.format_index(escape="latex", axis=1)
          .format(subset="App. Tarifaires J (%)", precision=0)
          .format(subset='App. Tarif. J-7 (%)', precision=0)
          .format(subset='Tarifs Vus J (%)', precision=0)
          .format(subset='Tarifs Vus J-7 (%)', precision=0)
          .format(subset='Contrats J (%)', precision=0)
          .format(subset='Contrats J-7 (%)', precision=0)
          .format_index(escape="latex", axis=0) #precision
          .hide(level=0, axis=0))
    #def rating_color(v):
     #   if v == "Buy": color = "#33FF85"
      #  elif v == "Sell": color = "#FF5933"
       # else: color = "#FFDD33"
        #return f"color: {color}; font-weight: bold;"
    (styler.background_gradient(cmap="Oranges", subset='Tarifs Vus J (%)'))
    #.background_gradient(cmap="Oranges", subset="Diff Tx conv."))
    #styler.applymap_index()   lambda v: "rotatebox:{45}--rwrap--latex;", level=2, axis=1)
    with open(r'/Users/marouan.korfed/Desktop/Pricing/Dashboard_LatexV2/Reporting_MIX_'+f'{variable}_'+f'{date1}_'+f'{type}''.tex', "w") as f:
      print(styler.to_latex(
        caption="Répartition Mix - "+f'{variable}',
        environment ='table*',
        clines="skip-last;data",
        convert_css=True,
        position_float="centering",
        multicol_align="|c|",
        hrules=True,
    )  , file=f)
    
    return df_total

In [493]:
date1 ="2023-06-16"
date2 ="2023-06-09"
df = df_organique

In [494]:
ReportingRepartitionMIX(date1,date2,df,'organique', "policy_risk_profile")
ReportingRepartitionMIX(date1,date2,df,'organique', "categorie_md_age")
ReportingRepartitionMIX(date1,date2,df,'organique', "categorie_vh_age")
ReportingRepartitionMIX(date1,date2,df,'organique', "categorie_nb_sinistres")
ReportingRepartitionMIX(date1,date2,df,'organique', "categorie_md_license_seniority")
ReportingRepartitionMIX(date1,date2,df,'organique', "categorie_md_crm_coefficient")

,categorie_md_crm_coefficient,App. Tarifaires J (%),App. Tarif. J-7 (%),Tarifs Vus J (%),Tarifs Vus J-7 (%),Contrats J (%),Contrats J-7 (%)
0,0.5,9.0,9.0,9.0,9.0,5.0,10.0
1,0.51-0.6,5.0,3.0,4.0,3.0,5.0,2.0
2,0.61-0.7,4.0,5.0,4.0,5.0,7.0,5.0
3,0.71-0.8,9.0,7.0,10.0,8.0,10.0,3.0
4,0.81-0.9,9.0,9.0,9.0,9.0,7.0,8.0
5,0.91-0.99,7.0,7.0,7.0,6.0,12.0,10.0
6,1,53.0,57.0,57.0,60.0,54.0,63.0
7,1+,4.0,4.0,0.0,0.0,0.0,0.0
8,Total,100.0,100.0,100.0,100.0,100.0,100.0


In [495]:
ReportingRepartitionMIX(date1,date2,df_lelynx,'lelynx', "policy_risk_profile")
ReportingRepartitionMIX(date1,date2,df_lelynx,'lelynx', "categorie_md_age")
ReportingRepartitionMIX(date1,date2,df_lelynx,'lelynx', "categorie_vh_age")
ReportingRepartitionMIX(date1,date2,df_lelynx,'lelynx', "categorie_nb_sinistres")
ReportingRepartitionMIX(date1,date2,df_lelynx,'lelynx', "categorie_md_license_seniority")
ReportingRepartitionMIX(date1,date2,df_lelynx,'lelynx', "categorie_md_crm_coefficient")

,categorie_md_crm_coefficient,App. Tarifaires J (%),App. Tarif. J-7 (%),Tarifs Vus J (%),Tarifs Vus J-7 (%),Contrats J (%),Contrats J-7 (%)
0,0.5,27.0,29.0,30.0,34.0,0.0,0.0
1,0.51-0.6,7.0,7.0,8.0,7.0,0.0,0.0
2,0.61-0.7,6.0,6.0,7.0,6.0,0.0,17.0
3,0.71-0.8,11.0,10.0,11.0,10.0,0.0,0.0
4,0.81-0.9,12.0,10.0,10.0,10.0,9.0,0.0
5,0.91-0.99,8.0,6.0,8.0,6.0,27.0,17.0
6,1,27.0,29.0,26.0,27.0,64.0,67.0
7,1+,3.0,2.0,0.0,0.0,0.0,0.0
8,Total,100.0,100.0,100.0,100.0,100.0,100.0


In [430]:
ReportingRepartitionMIX(date1,date2,db,'total', "policy_risk_profile")
ReportingRepartitionMIX(date1,date2,db,'total', "categorie_md_age")
ReportingRepartitionMIX(date1,date2,db,'total', "categorie_vh_age")
ReportingRepartitionMIX(date1,date2,db,'total', "categorie_nb_sinistres")
ReportingRepartitionMIX(date1,date2,db,'total', "categorie_md_license_seniority")
ReportingRepartitionMIX(date1,date2,db,'total', "categorie_md_crm_coefficient")

,categorie_md_crm_coefficient,App. Tarifaires J (%),App. Tarif. J-7 (%),Tarifs Vus J (%),Tarifs Vus J-7 (%),Contrats J (%),Contrats J-7 (%)
0,0.5,24.0,26.0,27.0,29.0,4.0,9.0
1,0.51-0.6,7.0,7.0,8.0,7.0,4.0,1.0
2,0.61-0.7,6.0,5.0,6.0,6.0,6.0,6.0
3,0.71-0.8,10.0,9.0,11.0,9.0,8.0,3.0
4,0.81-0.9,11.0,10.0,10.0,10.0,8.0,7.0
5,0.91-0.99,8.0,6.0,8.0,6.0,15.0,10.0
6,1,31.0,34.0,31.0,33.0,56.0,63.0
7,1+,3.0,2.0,0.0,0.0,0.0,0.0
8,Total,100.0,100.0,100.0,100.0,100.0,100.0
